In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [9]:
customersPath = "Customers.txt"            # cannot be stored in a variable
itemsCatalogPath = "ItemsCatalog.txt"      # can be stored in a variable
purchasesPath = "Purchases.txt"            # cannot be stored in a variable
customersRDD = sc.textFile(customersPath)
itemsRDD = sc.textFile(itemsCatalogPath)
purchasesRDD = sc.textFile(purchasesPath).cache()
output1 = "outPart1"
output2 = "outPart2"

In [10]:
threshold = 1000

In [11]:
def filterPurchases21_22(l):
    date = str(l.split(',')[0])
    return date.startswith("2021") or date.startswith("2022")

def mapPurchases21_22(l):
    dateYear = str(l.split(',')[0].split('/')[0])
    productId = str(l.split(',')[2])
    productIdYear = productId + "_" + dateYear
    return (productIdYear, 1)

In [12]:
productsIdYear2122 = purchasesRDD\
                    .filter(filterPurchases21_22)\
                    .map(mapPurchases21_22)\
                    .reduceByKey(lambda v1,v2: v1+v2)\
                    .filter(lambda elem: elem[1] >= threshold)\
                    .map(lambda elem: elem[0].split("_")[0])\
                    .distinct()

In [13]:
productsIdYear2122.collect()

['itemId100', 'itemId102']

In [27]:
# PART2
def filterItemsBefore20(l):
    date = int(l.split(',')[3].split('/')[0])
    return date < 2020

def mapItemCategory(l):
    itemId = str(l.split(',')[0])
    category = str(l.split(',')[2])
    return (itemId, category)

itemCategoryBefore20 = itemsRDD\
            .filter(filterItemsBefore20)\
            .map(mapItemCategory)\
            .cache()

In [28]:
itemCategoryBefore20.collect()

[('itemId1', 'Category9'),
 ('itemId2', 'Category2'),
 ('itemId3', 'Category1'),
 ('itemId4', 'Category4'),
 ('itemId5', 'Category4'),
 ('itemId6', 'Category9'),
 ('itemId7', 'Category1'),
 ('itemId8', 'Category0'),
 ('itemId9', 'Category0'),
 ('itemId10', 'Category8'),
 ('itemId11', 'Category2'),
 ('itemId12', 'Category6'),
 ('itemId13', 'Category0'),
 ('itemId14', 'Category8'),
 ('itemId15', 'Category1'),
 ('itemId16', 'Category6'),
 ('itemId17', 'Category2'),
 ('itemId18', 'Category4'),
 ('itemId19', 'Category9'),
 ('itemId20', 'Category2'),
 ('itemId21', 'Category6'),
 ('itemId22', 'Category0'),
 ('itemId23', 'Category3'),
 ('itemId24', 'Category7'),
 ('itemId25', 'Category2'),
 ('itemId26', 'Category2'),
 ('itemId27', 'Category0'),
 ('itemId28', 'Category7'),
 ('itemId29', 'Category5'),
 ('itemId30', 'Category3'),
 ('itemId31', 'Category9'),
 ('itemId32', 'Category1'),
 ('itemId33', 'Category0'),
 ('itemId34', 'Category8'),
 ('itemId35', 'Category5'),
 ('itemId36', 'Category4'),
 

In [84]:
def filterPurchases20(l):
    date = str(l.split(',')[0])
    return date.startswith("2020")

def mapPurchasesItemIDMUserId20(l):
    dateMonth = str(l.split(',')[0].split('/')[1])
    productId = str(l.split(',')[2])
    userId = str(l.split(',')[1])
    return (productId, (dateMonth, userId))

# ID1_01, User20
products20 = purchasesRDD\
        .filter(filterPurchases20)\
        .map(mapPurchasesItemIDMUserId20)

productsId20 = products20\
            .keys()\
            .distinct()
                    
products20WithMoreThan2Month = products20\
                        .join(itemCategoryBefore20)\
                        .map(lambda elem: (elem[0]+"_"+elem[1][0][0], elem[1][0][1]))\
                        .distinct()\
                        .map(lambda elem: (elem[0], 1))\
                        .reduceByKey(lambda v1, v2: v1 + v2)\
                        .filter(lambda elem: (elem[1] >= 10))\
                        .map(lambda elem: (elem[0].split("_")[0], 1))\
                        .reduceByKey(lambda v1, v2: v1 + v2)\
                        .filter(lambda elem: (elem[1] > 10))

In [85]:
productsId20.collect()

['itemId100', 'itemId200', 'itemId102', 'itemId201']

In [86]:
products20WithMoreThan2Month.collect()

[('itemId100', 12), ('itemId102', 12), ('itemId200', 12)]

In [87]:
products20WithMoreLessOrEqual2Months = itemCategoryBefore20.subtractByKey(products20WithMoreThan2Month)

In [88]:
print(products20WithMoreLessOrEqual2Months.count())

997
